In [ ]:
import numpy as np
import pandas as pd

import sys

import glob
import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
def forward_csv(f, prefix):
    pos = 0
    while True:
        line = f.readline()
        if not line or line.startswith(prefix):
            f.seek(pos)
            return f
        pos += len(line.encode('utf-8'))

In [ ]:
def read_frame(filenames):
    dfs = [ pd.read_csv(forward_csv(open(fn), 'name,iter')) for fn in filenames ]
    df = pd.merge(pd.merge(dfs[0], dfs[1], on='name'), dfs[2], on='name')
    
    # cancel out outliers due to OS jitter such as timers, kernel threads, etc.
    df['cpu_time_min'] = df[['cpu_time_x', 'cpu_time_y', 'cpu_time']].min(axis=1)

    df['ns'] = np.floor(df['cpu_time_min'])

    df['name'] = df['name'].map(lambda x : x[0:x.index('/')])
    df['table'] = df['name'].map(lambda x : x[0:x.index('_')])
    df['hash_fn'] = df['name'].map(lambda x : x[x.index('_')+1:])

    df = df[['name', 'ns', 'table', 'hash_fn']].copy()
    
    return df

In [ ]:
df1 = read_frame(glob.glob('out/xeon-gold-6246/*.csv'))
df1['tag'] = 'xeon-gold-6246'
df2 = read_frame(glob.glob('out/atom-C3758/*.csv'))
df2['tag'] = 'atom-C3758'

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
df

In [ ]:
sns.boxenplot(y='table', hue='hash_fn', x='ns', data=df1, orient='h')

In [ ]:
sns.boxenplot(x='table', hue='hash_fn', y='ns', data=df2)

In [ ]:
df

In [ ]:
sns.set_style("darkgrid")
#sns.set_style("dark")
#plt.style.use("dark_background")
#sns.set_style("whitegrid")
# height=3.5
sns.catplot(kind='boxen', x='table', hue='hash_fn', y='ns', col='tag', data=df, height=3.45)
plt.savefig('perfect-hashing-bplot.svg', format='svg')
plt.show()